<a href="https://colab.research.google.com/github/RuwaAbey/Finaly_Year_Project_G09/blob/main/Self_attention_for_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import math

In [5]:
L, d_k, d_v = 4, 8, 8
q = np.random.randn(L, d_k)
k = np.random.randn(L, d_k)
v = np.random.randn(L, d_v)

In [6]:
print("Q\n", q)
print("K\n", k)
print("V\n", v)

Q
 [[-1.46432704  0.20471281 -0.32898122  0.89483735  0.2242479   0.03151882
  -0.29206521 -0.46967375]
 [ 1.10724791  0.34299864 -0.97608017  0.67612317 -0.90973631  0.2546449
   0.34335394  0.21816381]
 [ 0.52143217  2.07134144  0.79366991 -0.94910141  0.3132216   0.90215894
   0.12000727  1.18297572]
 [ 1.3669547   0.22876581 -0.98598411  0.96910186 -0.25485859 -1.49945397
  -1.17463663 -0.65297544]]
K
 [[-0.24670269  0.32929309  1.94359317 -0.14318404 -2.18338484  1.36615366
  -0.90661051 -0.20062246]
 [-0.58742953  0.59282429  0.32258675 -1.44393185  1.11899284 -0.6023521
  -0.37999452  0.01818678]
 [ 1.88333998 -1.66048334 -0.60445395  0.8263982   0.86080805  0.01275883
   1.13997316 -1.94347608]
 [-0.26982031 -1.31406699  1.18463529  0.51031721 -0.56188925  1.2418082
   0.03442553 -0.36791069]]
V
 [[ 1.25869732 -1.55019724 -1.35622277  0.40265317 -1.43280593 -0.13594235
  -0.92121947 -1.02116546]
 [-0.67461386 -0.36638122  0.44740676 -0.07170293  1.46450041 -0.30134998
  -0.6201

So here if I consider a sentence "My name is Ruwa" each work has got Q,K,V vector.

So that size of each of the Q, K, V vectors is 4 x 8

4 - number of words
8 - number of elements used to represent the work



**Self attention**

$\text{self-attention} = \text{softmax} \left( \frac{QK^T}{\sqrt{d_k}} + M \right) V$


In [7]:
np.matmul(q, k.T)

array([[-0.42641157, -0.08227377, -1.38611215,  0.26890831],
       [-0.17499509, -3.03611772,  1.85208197, -0.80179638],
       [ 2.43438041,  2.3310898 , -5.6026123 , -1.89349338],
       [-2.61310299, -1.3322598 ,  3.28289693, -2.86196072]])

This matrix represent how much each work is related other words.

       My        | name            | is |Ruwa

My   |-1.14929296|  2.62199025     |

name |

is   |

Ruwa |

In [9]:
#why we need sqrt(d_k) in denominator
q.var(), k.var(), np.matmul(q, k.T).var() #checking varainces of Q,K and Q.KT

(0.7308643072181876, 1.0855078221478214, 5.194239102626842)

In [10]:
scaled = np.matmul(q,k.T) / math.sqrt(d_k)
q.var(), k.var(), scaled.var()

(0.7308643072181876, 1.0855078221478214, 0.649279887828355)

Note the reduction in variance of the procut

In [11]:
scaled

array([[-0.15075926, -0.02908817, -0.49006465,  0.09507344],
       [-0.06187011, -1.07342971,  0.65480986, -0.28347783],
       [ 0.86068345,  0.8241647 , -1.98082258, -0.669451  ],
       [-0.92387142, -0.47102497,  1.16067934, -1.01185592]])

**Masking**

* This is to ensure words don't get context from words generated in the future

* Not required in the encoders, but required int the decoders

In [13]:
mask = np.tril(np.ones((L,L)))

In [14]:
mask

array([[1., 0., 0., 0.],
       [1., 1., 0., 0.],
       [1., 1., 1., 0.],
       [1., 1., 1., 1.]])

In [16]:
mask[mask == 0] = -np.infty
mask[mask == 1] = 0

In [17]:
mask

array([[  0., -inf, -inf, -inf],
       [  0.,   0., -inf, -inf],
       [  0.,   0.,   0., -inf],
       [  0.,   0.,   0.,   0.]])

In [18]:
scaled + mask

array([[-0.15075926,        -inf,        -inf,        -inf],
       [-0.06187011, -1.07342971,        -inf,        -inf],
       [ 0.86068345,  0.8241647 , -1.98082258,        -inf],
       [-0.92387142, -0.47102497,  1.16067934, -1.01185592]])

why negative infinty?

* because of softmax operation


**Softmax**  

$
\text{softmax}(x_i) = \frac{e^{x_i}}{\sum_j e^{x}_j}
$


In [19]:
def softmax(x):
  return (np.exp(x).T / np.sum(np.exp(x), axis = 1)).T

In [20]:
attention = softmax(scaled + mask)

In [21]:
attention

array([[1.        , 0.        , 0.        , 0.        ],
       [0.73332526, 0.26667474, 0.        , 0.        ],
       [0.49444301, 0.4767123 , 0.02884469, 0.        ],
       [0.08673376, 0.13641335, 0.69742428, 0.07942861]])

In [22]:
attention_without_mask = softmax(scaled)

In [23]:
attention_without_mask

array([[0.24269882, 0.27409982, 0.17286585, 0.3103355 ],
       [0.23738846, 0.08632664, 0.48608224, 0.19020266],
       [0.44663109, 0.43061491, 0.02605545, 0.09669856],
       [0.08673376, 0.13641335, 0.69742428, 0.07942861]])

In [24]:
new_v = np.matmul(attention, v)
new_v

array([[ 1.25869732, -1.55019724, -1.35622277,  0.40265317, -1.43280593,
        -0.13594235, -0.92121947, -1.02116546],
       [ 0.74313205, -1.2345034 , -0.87524033,  0.27615438, -0.6601675 ,
        -0.18005239, -0.84094296, -0.69312885],
       [ 0.30272423, -0.92774945, -0.43402808,  0.13468437, -0.02038519,
        -0.18171859, -0.71160133, -0.36342005],
       [ 0.071845  ,  0.08823529,  0.64389911, -0.76105317, -0.20687973,
         0.77177021,  0.80843675,  1.03463344]])

In [25]:
v

array([[ 1.25869732, -1.55019724, -1.35622277,  0.40265317, -1.43280593,
        -0.13594235, -0.92121947, -1.02116546],
       [-0.67461386, -0.36638122,  0.44740676, -0.07170293,  1.46450041,
        -0.30134998, -0.6201917 ,  0.20893456],
       [ 0.06818796,  0.46432045,  0.80647389, -1.0477841 , -0.34979269,
         1.0107372 ,  1.37086607,  1.4521356 ],
       [ 0.08994057, -0.64409209,  1.73794655, -0.69805435, -0.48383808,
         1.50772494,  0.21232556,  1.03170477]])

comapare before attention and after attention with v

In [26]:
def softmax(x):
  return (np.exp(x).T/np.sum(np.exp(x), axis = -1)).T

def scaled_dot_product_attention(q,k,v,mask=None):
  d_k = q.shape[-1]
  scaled = np.matmul(q,k.T) / math.sqrt(d_k)
  if mask is not None:
    scaled = scaled + mask
  attention = softmax(scaled)
  out = np.matmul(attention, v)
  return out, attention


In [27]:
values, attention = scaled_dot_product_attention(q,k,v, mask=None)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("New", values)
print("Attention", attention)

Q
 [[-1.46432704  0.20471281 -0.32898122  0.89483735  0.2242479   0.03151882
  -0.29206521 -0.46967375]
 [ 1.10724791  0.34299864 -0.97608017  0.67612317 -0.90973631  0.2546449
   0.34335394  0.21816381]
 [ 0.52143217  2.07134144  0.79366991 -0.94910141  0.3132216   0.90215894
   0.12000727  1.18297572]
 [ 1.3669547   0.22876581 -0.98598411  0.96910186 -0.25485859 -1.49945397
  -1.17463663 -0.65297544]]
K
 [[-0.24670269  0.32929309  1.94359317 -0.14318404 -2.18338484  1.36615366
  -0.90661051 -0.20062246]
 [-0.58742953  0.59282429  0.32258675 -1.44393185  1.11899284 -0.6023521
  -0.37999452  0.01818678]
 [ 1.88333998 -1.66048334 -0.60445395  0.8263982   0.86080805  0.01275883
   1.13997316 -1.94347608]
 [-0.26982031 -1.31406699  1.18463529  0.51031721 -0.56188925  1.2418082
   0.03442553 -0.36791069]]
V
 [[ 1.25869732 -1.55019724 -1.35622277  0.40265317 -1.43280593 -0.13594235
  -0.92121947 -1.02116546]
 [-0.67461386 -0.36638122  0.44740676 -0.07170293  1.46450041 -0.30134998
  -0.6201

In [28]:
values, attention = scaled_dot_product_attention(q,k,v, mask=mask)
print("Q\n", q)
print("K\n", k)
print("V\n", v)
print("New", values)
print("Attention", attention)

Q
 [[-1.46432704  0.20471281 -0.32898122  0.89483735  0.2242479   0.03151882
  -0.29206521 -0.46967375]
 [ 1.10724791  0.34299864 -0.97608017  0.67612317 -0.90973631  0.2546449
   0.34335394  0.21816381]
 [ 0.52143217  2.07134144  0.79366991 -0.94910141  0.3132216   0.90215894
   0.12000727  1.18297572]
 [ 1.3669547   0.22876581 -0.98598411  0.96910186 -0.25485859 -1.49945397
  -1.17463663 -0.65297544]]
K
 [[-0.24670269  0.32929309  1.94359317 -0.14318404 -2.18338484  1.36615366
  -0.90661051 -0.20062246]
 [-0.58742953  0.59282429  0.32258675 -1.44393185  1.11899284 -0.6023521
  -0.37999452  0.01818678]
 [ 1.88333998 -1.66048334 -0.60445395  0.8263982   0.86080805  0.01275883
   1.13997316 -1.94347608]
 [-0.26982031 -1.31406699  1.18463529  0.51031721 -0.56188925  1.2418082
   0.03442553 -0.36791069]]
V
 [[ 1.25869732 -1.55019724 -1.35622277  0.40265317 -1.43280593 -0.13594235
  -0.92121947 -1.02116546]
 [-0.67461386 -0.36638122  0.44740676 -0.07170293  1.46450041 -0.30134998
  -0.6201

No word get any attention form the words that come after it. (in this created mask)